In [3]:
import pandas as pd

# 깃허브에 있는 원시(raw) 파일 경로
train_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
test_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt'

# URL에서 바로 데이터를 DataFrame으로 불러오기
# \t는 tap으로 분리되어 있다는 의미입니다.
try:
    train_df = pd.read_csv(train_url, sep='\t')
    test_df = pd.read_csv(test_url, sep='\t')
    print("파일을 성공적으로 불러왔습니다!")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")


# 데이터가 잘 불러와졌는지 상위 5개만 확인
print("--- 훈련 데이터 (train_df) ---")
print(train_df.head())

# print("\n--- 테스트 데이터 (test_df) ---")
# print(test_df.head())

파일을 성공적으로 불러왔습니다!
--- 훈련 데이터 (train_df) ---
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1


In [4]:
# 1. 비어있는 값(NaN)이 있는지 확인
print(train_df.isnull().sum())

# 2. 비어있는 값이 있는 행(row)을 제거
train_df = train_df.dropna()

# 3. 우리가 필요한 'document' 컬럼만 확인
print("\n정제 후 데이터 샘플:")
print(train_df['document'].head())

id          0
document    5
label       0
dtype: int64

정제 후 데이터 샘플:
0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object


In [5]:
from sentence_transformers import SentenceTransformer

# 한국어 문장 임베딩에 특화된 모델을 불러옵니다.
model = SentenceTransformer('jhgan/ko-sbert-nli')

print("\n임베딩 모델을 성공적으로 불러왔습니다!")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\sec\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sec\.cache\huggingface\hub\models--jhgan--ko-sbert-nli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


임베딩 모델을 성공적으로 불러왔습니다!


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
# !pip install sentence-transformers chromadb pandas

import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb

# 1. 데이터 준비 (로딩, 정제, 축소)
train_url = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
train_df = pd.read_csv(train_url, sep='\t')
train_df = train_df.dropna()
target_df = train_df.head(10000)

# 2. 임베딩 실행
# (참고: show_progress_bar=True 때문에 임베딩 진행률 막대는 표시됩니다)
model = SentenceTransformer('jhgan/ko-sbert-nli')
documents = target_df['document'].tolist()
embeddings = model.encode(documents, show_progress_bar=True)

# 3. ChromaDB에 저장
client = chromadb.Client()
collection = client.get_or_create_collection(name="movie_reviews")
ids = [str(i) for i in range(len(documents))]
collection.add(
    documents=documents,
    embeddings=embeddings,
    ids=ids
)

# 4. 검색 및 결과 출력
query_text = "인생 최고의 영화"
query_embedding = model.encode([query_text])
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

# --- 최종 검색 결과 ---
print(f"\n[검색어] {query_text}")
print("-----------")
for i, doc in enumerate(results['documents'][0]):
    print(f"{i+1}위: {doc}")

Batches:   0%|          | 0/313 [00:00<?, ?it/s]